In [ ]:
import requests
from bs4 import BeautifulSoup

# 1. Obtener la URL
url = "https://books.toscrape.com/"
respuesta_obtenida = requests.get(url)
html_obtenido = respuesta_obtenida.text  # HTML como string

# 2. Parsear ese HTML con BeautifulSoup
soup = BeautifulSoup(html_obtenido, 'html.parser')  # <- ¡ESTO FALTABA!

# 3. Buscar el primer título <h3>
primer_h3 = soup.find('h3')

# 4. Mostrar el contenido
print(primer_h3)         # Muestra todo el tag <h3>...</h3>
print(primer_h3.text)    # Muestra solo el texto (título del libro)
print(soup.h3.text)      # Equivalente

# 5. Trayendo todos los <h3>

h3_todos = soup.find_all('h3')
print(h3_todos) #lista con todos los elementos de h3

# 6. Iteracion de todos los elementos poniendo solo texto 
contador = 0
for unoporuno in h3_todos:
    contador+=1

    print(contador, unoporuno.text)
    






In [1]:
import requests
from bs4 import BeautifulSoup
libros = None


base_url = "https://books.toscrape.com/catalogue/page-{}.html"


#funcion para recorrer todas las paginas 

for page in range(1, 2):  # Hay 50 páginas en total
    url = base_url.format(page)
    respuesta = requests.get(url)
    soup = BeautifulSoup(respuesta.text, 'html.parser')

    # Aquí haces scraping de los libros de esa página
    libros = soup.find_all('article', class_='product_pod')
    print(f"📄 Página {page} -> {len(libros)} libros encontrados")

# recorrer_paginas()


#   

for libro in libros:
    titulo = libro.h3.a['title']
    precio = libro.find('p', class_='price_color').text
    disponibilidad = libro.find('p', class_='instock availability').text.strip()

    # Enlace al detalle del libro (relativo -> convertir a absoluto)
    enlace_relativo = libro.h3.a['href']
    print(enlace_relativo)
    enlace_absoluto = "https://books.toscrape.com/catalogue/" + enlace_relativo

    #print(f"📚 {titulo} | 💵 {precio} | 📦 {disponibilidad} | 🔗 {enlace_absoluto}")
    
    
    
detalle = requests.get(enlace_absoluto)
soup_detalle = BeautifulSoup(detalle.text, 'html.parser')

# Categoría: está en un breadcrumb (<ul class="breadcrumb">)
categoria = soup_detalle.find('ul', class_='breadcrumb').find_all('a')[-1].text

#print(f"📚 {titulo} | 💵 {precio} | 📦 {disponibilidad} | 📂 {categoria}")

    
    





📄 Página 1 -> 20 libros encontrados
a-light-in-the-attic_1000/index.html
tipping-the-velvet_999/index.html
soumission_998/index.html
sharp-objects_997/index.html
sapiens-a-brief-history-of-humankind_996/index.html
the-requiem-red_995/index.html
the-dirty-little-secrets-of-getting-your-dream-job_994/index.html
the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html
the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html
the-black-maria_991/index.html
starving-hearts-triangular-trade-trilogy-1_990/index.html
shakespeares-sonnets_989/index.html
set-me-free_988/index.html
scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html
rip-it-up-and-start-again_986/index.html
our-band-could-be-your-life-scenes-from-the-american-indie-underground-1981-1991_985/index.html
olio_984/index.html
mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html
libertarianism-for-beginners_982/in

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from concurrent.futures import ThreadPoolExecutor

def get_with_retries(url, headers=None, max_retries=3, timeout=5):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            if response.status_code == 200:
                return response
        except requests.exceptions.RequestException as e:
            print(f'Intento {attempt+1} fallido para {url}: {e}')
            time.sleep(2)
    print(f'No se pudo acceder a {url} después de {max_retries} intentos.')
    return None

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
base_url = "https://books.toscrape.com/"

def get_google_books_authors(titulo):
    titulo_api = titulo.replace(" ", "%")
    api_url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{titulo_api}"
    try:
        resp_api = requests.get(api_url, timeout=10)
        data_api = resp_api.json()
        if "items" in data_api and len(data_api["items"]) > 0:
            return data_api["items"][0]["volumeInfo"].get("authors", [])
    except:
        pass
    return []

def process_book(libro, nombre_cat):
    titulo = libro.h3.a['title']
    precio = libro.find('p', class_='price_color').text
    disponibilidad = libro.find('p', class_='instock availability').text.strip()
    rating = libro.p['class'][1] if len(libro.p['class']) > 1 else 'None'
    
    enlace_rel = libro.h3.a['href']
    if enlace_rel.startswith('../'):
        enlace_rel = enlace_rel.replace('../', '')
    enlace_abs = base_url + 'catalogue/' + enlace_rel
    
    # Obtener detalles de la página del libro
    detalle = get_with_retries(enlace_abs, headers=headers)
    if detalle is None:
        return None
    
    soup_detalle = BeautifulSoup(detalle.text, 'html.parser')
    
    try:
        categoria = soup_detalle.find('ul', class_='breadcrumb').find_all('a')[-1].text.strip()
    except:
        categoria = nombre_cat
    
    try:
        stock = soup_detalle.find('p', class_='instock availability').text.strip()
    except:
        stock = ''
    
    # Obtener autores de Google Books (en paralelo)
    autores = get_google_books_authors(titulo)
    
    return {
        'Título': titulo,
        'Autores': autores,
        'Precio': precio,
        'Disponibilidad': disponibilidad,
        'Rating': rating,
        'Categoría': categoria,
        'Stock': stock,
        'URL': enlace_abs
    }

def main():
    # Obtener categorías
    respuesta = get_with_retries(base_url, headers=headers)
    if respuesta is None:
        raise Exception('No se pudo obtener la página principal')
    
    soup = BeautifulSoup(respuesta.text, 'html.parser')
    categorias = {
        a.text.strip(): base_url + a['href'] 
        for a in soup.select('ul.nav.nav-list li a') 
        if a['href'] != 'catalogue/category/books_1/index.html'
    }
    
    print(f"Categorías encontradas: {len(categorias)}")
    
    libros_data = []
    contador = 0
    
    for nombre_cat, url_cat in categorias.items():
        page = 1
        while True:
            url_pagina = url_cat if page == 1 else url_cat.replace('index.html', f'page-{page}.html')
            
            resp_cat = get_with_retries(url_pagina, headers=headers)
            if resp_cat is None:
                break
                
            soup_cat = BeautifulSoup(resp_cat.text, 'html.parser')
            libros = soup_cat.find_all('article', class_='product_pod')
            
            if not libros:
                break
            
            # Procesar libros en paralelo (limitado a 3 hilos para no saturar)
            with ThreadPoolExecutor(max_workers=3) as executor:
                futures = [executor.submit(process_book, libro, nombre_cat) for libro in libros]
                
                for future in futures:
                    libro_info = future.result()
                    if libro_info:
                        libros_data.append(libro_info)
                        contador += 1
                        print(contador, libro_info)
            
            # Verificar si hay siguiente página
            next_page = soup_cat.find('li', class_='next')
            if not next_page:
                break
                
            page += 1
            time.sleep(0.5)  # Pequeña pausa entre páginas
    
    return libros_data

if __name__ == "__main__":
    start_time = time.time()
    data = main()
    end_time = time.time()
    print(f"\nTiempo total de ejecución: {end_time - start_time:.2f} segundos")
    print(f"Total de libros obtenidos: {len(data)}")


Categorías encontradas: 50
1 {'Título': "It's Only the Himalayas", 'Autores': ['S. Bedford'], 'Precio': 'Â£45.17', 'Disponibilidad': 'In stock', 'Rating': 'Two', 'Categoría': 'Travel', 'Stock': 'In stock (19 available)', 'URL': 'https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html'}
2 {'Título': 'Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond', 'Autores': ['Rick Antonson'], 'Precio': 'Â£49.43', 'Disponibilidad': 'In stock', 'Rating': 'Four', 'Categoría': 'Travel', 'Stock': 'In stock (15 available)', 'URL': 'https://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html'}
3 {'Título': 'See America: A Celebration of Our National Parks & Treasured Sites', 'Autores': ['Creative Action Network'], 'Precio': 'Â£48.87', 'Disponibilidad': 'In stock', 'Rating': 'Three', 'Categoría': 'Travel', 'Stock': 'In stock (14 available)', 'URL': 'https://books.toscrape.com/catalogue/see-america-a-celebration

KeyboardInterrupt: 